<a href="https://colab.research.google.com/github/freguti/ProgettoDataSpaces/blob/main/Progetto_DataSpaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Analysis of "Predicting Pulsar Star"**

Simone Soncin - s263094

Scaletta:

- Analizzare dataset originale, fornendo informazioni sulla distribuzione dei dati e delle classi
- Bilanciare il train dataset  
- eseguire algoritmi di classificazione sul dataset modificato
- ...

##**Indice**

 

1. [Introduzione](#Introduzione)
2. Analisi Dataset
3. [PCA](#PCA)
4. Bilanciamento Datadet
5. Classificazione
  *   Primo classificatore
  *   Secondo classificatore
6. Conclusioni










In [2]:
#Show/Hide code button
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">Il codice è stato nascosto per permettere una lettura più fluida. Per mostrarlo cliccare qui<br><input type="submit" value="Abilita/Disabilita codice"></form>''')

<a name="introduzione"></a>
##**Introduzione**
In questa tesina mi dedicherò ad analizzare un dataset, manipolare i dati per utilizzarli in maniera corretta e successivamente applicare dei modelli predittivi, identificando quale si comporta meglio nel nostro caso di studio.

Le stelle pulsar sono delle stelle di neutroni che emettono un potente segnale radio dai poli misurabile dalla terra. Una stella pulsar ruota ad altissima velocità, rendendo questo segnale radio misurabile a tratti e fornendogli un pattern periodico che varia per ogni stella pulsar.

Il dataset che utilizzerò è disponibile al link: https://www.kaggle.com/colearninglounge/predicting-pulsar-starintermediate?select=pulsar_data_train.csv 

Questo dataset contiene una serie di parametri continui discretizzati, i cui primi 4 sono derivati dal "integrated pulse profile", mentre gli ultimi 4 parametri sono ottenuti in maniera analoga dalla "DM-SNR curve".

- Integrated profile: è ottenuto dal segnale periodico emesso dalla pulsar ed è univoco per ognuna di esse. Per questo motivo è utilizzato per identificarle a discapito del loro nome. 
- DM-SNR Curve: Dispersion Measure of the Signal to Noise Ratio in poche parole è la densità degli elettroni liberi lungo la linea ottica.

I nostri parametri sono stati ottenuti da questi due dati, applicando rispettivamente la media, la deviazione standard, la curtosi (indica quanto sono spesse le code della distribuzione) e la simmetria statistica (asimmetria della distribuzione di probabilità rispetto alla sua media).

Ecco la lista dettagliata dei parametri:
- **Mean of the integrated profile**
- **Standard deviation of the integrated profile**
- **Excess kurtosis of the integrated profile**
- **Skewness of the integrated profile**
- **Mean of the DM-SNR curve**
- **Standard deviation of the DM-SNR curve**
- **Excess kurtosis of the DM-SNR curve**
- **Skewness of the DM-SNR curve**

Infine è presente un ultimo parametro discreto che indica se il segnale appartiene ad una pulsar (target_class = 1) o no (target_class = 0).

###**Strumenti Utilizzati**
TODO
- Grafici: Per i grafici è stata utilizzata la libreria chart studio, che permette di creare dei grafici interattivi in maniera semplice ed intuitiva. Questo strumento ha il limite che non consente un upload di dati superiore ai 500Kb. Quando ho potuto ho provveduto a separare i grafici a coppie, ma nel caso della matrice di correlazione e del dendogramma ho optato per uno strumento più rudimentale, DA DEFINIRE

##Utility

In [3]:
#Capture removes the cell's output
#%%capture
#Imports
import pandas as pd
import numpy as np
from google.colab import drive
!pip install -U imbalanced-learn
from imblearn.over_sampling import ADASYN
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
!pip install nbconvert
!pip install -q gwpy
import plotly
!pip install chart_studio
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from numpy import mean

#alla fine sarà da eseguire alla fine del notebook
#%%capture
#It's used to convert this notebook in HTML
#!jupyter nbconvert --to html /content/drive/MyDrive/DataSpaces/Progetto_DataSpaces.ipynb

     |████████████████████████████████| 215kB 13.1MB/s 
     |████████████████████████████████| 22.3MB 70.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3
     |████████████████████████████████| 1.4MB 11.0MB/s 
     |████████████████████████████████| 10.3MB 57.4MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
     |████████████████████████████████| 3.2MB 56.2MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 71kB 5.2MB/s 


In [4]:
#Parameters
%%capture
chart_studio.tools.set_credentials_file(username='freguti', api_key='4T2iyEX3eZP4aJ8PEJye')
HISTOGRAM_COLORS = {"pulsar" : "#388004",
                    "not_pulsar" : "#8B0000"}
drive.mount('/content/drive')
BOX_OPACITY = 0.3

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWgyJzpsz1T0siWbg1Lt3IxyqSolLLIoah-hEKDCeSvEYDlcvsJFdIA


In [84]:
#Graph utility
"""
def oneFeatureHistogram(dataset,feature,title) :
  x = dataset[feature]
  image, plot = plt.subplots(figsize=(16, 10))
  plot.hist(x)
  for rect in plot.patches:
    height = rect.get_height()
    if height > 0:
      plot.annotate(f'{int(height)}', xy=(rect.get_x()+rect.get_width()/2, height), 
                xytext=(0, 5), textcoords='offset points', ha='center', va='bottom') 
  plt.xlabel(feature)
  plt.ylabel('value')
  plt.title(title)
  plt.plot()
  #plt.draw()
  plt.show()
  """


def histogram_bar(tipo, data, col, visible=True, norm = ""):
    
    color = HISTOGRAM_COLORS[tipo]
    
    data_16 = data.astype(np.float16 ,copy=True)
    
    return go.Histogram(
        x = data_16[col],
        name = tipo,
        marker = dict(color = color),
        visible = visible,
        opacity = 1,
        histnorm=norm
    )

#fare l'istogramma con chart studio https://plotly.com/python/getting-started-with-chart-studio/
def feature_histogram(dataset, title = "", feature = ""):

  trace0 = histogram_bar("not_pulsar", dataset[dataset['target_class'] == 0], feature)
  trace1 = histogram_bar("pulsar", dataset[dataset['target_class'] == 1], feature)
  
  data = [trace0, trace1]
  
  layout = dict(
      title=title,
      autosize=True,
      yaxis=dict(
          title='value',
          automargin=True,
      ),
      legend=dict(
          x=0,
          y=1,
      ),
      barmode='group',
      bargap=0.15,
      bargroupgap=0.1
  )
  fig = dict(data=data, layout=layout)
  return py.iplot(fig, filename=title)


def on_slider_change(sender,dataset):
  if sender['name'] == 'value':
    selected_option = sender['owner'].value
    print(selected_option)
    feature_histogram(dataset = dataset, title = "Distribuzione features", feature = selected_option)

def create_slider_controlled_histogram(col,pulsar,not_pulsar,first_plot):
  pulsar_plot = []
  not_pulsar_plot = []
  hist_features,pulsar_plot,not_pulsar_plot = prepare_set(dataset.columns,first_plot,pulsar,not_pulsar)
  active_index = 0
  print(hist_features)
  hist_not = [(histogram_bar('not_pulsar', not_pulsar_plot, col, False,'percent') if i != active_index
            else histogram_bar('not_pulsar', not_pulsar_plot, col, True,'percent'))
              for i, col in enumerate(hist_features)
              ]
  hist_pulsar = [(histogram_bar('pulsar', pulsar_plot, col, False,'percent') if i != active_index
            else histogram_bar('pulsar', pulsar_plot, col, True,'percent'))
              for i, col in enumerate(hist_features)
              ]

  total_data = hist_not + hist_pulsar
  number_of_features = len(hist_features)
  steps = []

  for i in range(number_of_features):
      step = dict(
          method = 'restyle',
          args = ['visible', [False] * number_of_features],
          label = hist_features[i],
      )
      step['args'][1][i] = True
      steps.append(step)
      
  sliders = [dict(
      active = active_index,
      currentvalue = dict(
          prefix = "Feature: ", 
          xanchor= 'center',
      ),
      pad = {"t": 50},
      steps = steps,
      len=1,
  )]

  layout = dict(
      sliders=sliders,
      autosize=True,
      yaxis=dict(
          title='valori',
          automargin=True,
      ),
      legend=dict(
          x=0,
          y=1,
      ),
  )

  fig = dict(data=total_data, layout=layout)

  return py.iplot(fig, filename='bar_slider')  

def create_box(type, data, col, visible=False):
   
    c = HISTOGRAM_COLORS[type]
    data_16 = data.astype(np.float16 ,copy=True)
    return go.Box(
        y = data_16[col],
        name = type,
        marker = dict(color = c),
        visible = visible,
        opacity = BOX_OPACITY,
    )

def create_slider_controlled_Boxplot(col,pulsar,not_pulsar,first_plot):
  pulsar_plot = []
  not_pulsar_plot = []
  box_features,pulsar_plot,not_pulsar_plot = prepare_set(dataset.columns,first_plot,pulsar,not_pulsar)
  active_index = 0
  print(box_features)

  box_not_pulsar = [(create_box('not_pulsar', not_pulsar_plot, col, False) if i != active_index
            else create_box('not_pulsar', not_pulsar_plot, col, True))
              for i, col in enumerate(box_features)
              ]
  box_pulsar = [(create_box('pulsar', pulsar_plot, col, False) if i != active_index
              else create_box('pulsar', pulsar_plot, col, True))
              for i, col in enumerate(box_features)
              ]

  data = box_not_pulsar + box_pulsar
  number_of_features = len(box_features)
  steps = []

  for i in range(number_of_features):
      step = dict(
          method = 'restyle',  
          args = ['visible', [False] * number_of_features],
          label = box_features[i],
      )
      step['args'][1][i] = True # Toggle i'th trace to "visible"
      steps.append(step)
      
  sliders = [dict(
      active = active_index,
      currentvalue = dict(
          prefix = "Feature: ", 
          xanchor= 'center',
      ),
      pad = {"t": 50},
      steps = steps,
      len=1,
  )]

  layout = dict(
      sliders=sliders,
      autosize=True,
      yaxis=dict(
          title='valori',
          automargin=True,
      ),
      legend=dict(
          x=0,
          y=1,
      ),
  )

  fig = dict(data=data, layout=layout)
  return py.iplot(fig, filename='box_slider')

def prepare_set(col,first_plot,pulsar,not_pulsar): 
  pulsar_plot = pulsar
  not_pulsar_plot = not_pulsar
  box_features = col.values.tolist()
  if first_plot == "mean":
    #to_drop = [box_features[1],box_features[2],box_features[3],box_features[5],box_features[6],box_features[7],box_features[8]]
    #pulsar_plot = pulsar.drop(to_drop,axis =1)
    #not_pulsar_plot = not_pulsar.drop(to_drop,axis =1)
    hist_features = [box_features[0],box_features[4]]
  if first_plot == "deviation":
    #to_drop = [box_features[0],box_features[2],box_features[3],box_features[4],box_features[6],box_features[7],box_features[8]]
    #pulsar_plot = pulsar.drop(to_drop,axis =1)
    #not_pulsar_plot = not_pulsar.drop(to_drop,axis =1)
    hist_features = [box_features[1],box_features[5]]
  if first_plot == "kurtosis":
    #to_drop = [box_features[0],box_features[1],box_features[3],box_features[4],box_features[5],box_features[7],box_features[8]]
    #pulsar_plot = pulsar.drop(to_drop,axis =1)
    #not_pulsar_plot = not_pulsar.drop(to_drop,axis =1)
    hist_features = [box_features[2],box_features[6]]
  if first_plot == "skewness":
    #to_drop = [box_features[0],box_features[1],box_features[2],box_features[4],box_features[5],box_features[6],box_features[8]]
    #pulsar_plot = pulsar.drop(to_drop,axis =1)
    #not_pulsar_plot = not_pulsar.drop(to_drop,axis =1)
    hist_features = [box_features[3],box_features[7]]
  return hist_features,pulsar_plot,not_pulsar_plot

##**Analisi Dataset**

Come prima cosa è necessario eseguire un'analisi del dataset, eseguendo eventualmente delle operazioni per migliorarlo e renderlo più adatto al nostro tipo di analisi.

La fase di analisi è la più importante, perchè permette di pulire il dataset migliorando l'analisi effettuata e ridurre la potenza di calcolo necessaria per eseguirla.



In [58]:

dataset = pd.read_csv('/content/drive/My Drive/DataSpaces/pulsar_stars.csv')
#print(dataset.info(memory_usage = "deep"))
for column in dataset:
 if dataset[column].dtype == 'float64':
  dataset[column]=pd.to_numeric(dataset[column], downcast='float')
  if dataset[column].min() <= np.finfo(np.float16).min or dataset[column].max() >= np.finfo(np.float16).max:
    raise CustomError("Range Exception")
 if dataset[column].dtype == 'int64':
  dataset[column]=pd.to_numeric(dataset[column], downcast='integer')
#print(dataset.info(memory_usage = "deep"))
print("il dataset ha %d record e %d features\n\nNumero di valori null per ogni feature:" % dataset.shape)
print(dataset.isnull().sum())

print("\nDa quel che si può vedere, non ci sono valori nulli all'interno del nostro dataset. Nel caso in cui ne avessimo trovato uno avremo avuto due possibilità:\n")
print("- Se i valori nulli non sono molti possiamo procedere eliminando i record che riportano il valore nullo;\n- Se il numero di questi valori nulli è elevato si può procedere alla sostituzione con dei valori di default\n")

print("ci sono 2 classi differenti.\n0 -> non è una pulsar\n1 -> è una pulsar")
X, y = make_classification(n_classes = 2, )
target_class = dataset['target_class'].value_counts()
print("numero di record per ogni classe:\n" + target_class.to_string() + "\n")

il dataset ha 17592 record e 9 features

Numero di valori null per ogni feature:
 Mean of the integrated profile                  0
 Standard deviation of the integrated profile    0
 Excess kurtosis of the integrated profile       0
 Skewness of the integrated profile              0
 Mean of the DM-SNR curve                        0
 Standard deviation of the DM-SNR curve          0
 Excess kurtosis of the DM-SNR curve             0
 Skewness of the DM-SNR curve                    0
target_class                                     0
dtype: int64

Da quel che si può vedere, non ci sono valori nulli all'interno del nostro dataset. Nel caso in cui ne avessimo trovato uno avremo avuto due possibilità:

- Se i valori nulli non sono molti possiamo procedere eliminando i record che riportano il valore nullo;
- Se il numero di questi valori nulli è elevato si può procedere alla sostituzione con dei valori di default

ci sono 2 classi differenti.
0 -> non è una pulsar
1 -> è una pulsar
numero 

La prima analisi che si dovrà effettuare è quella che permette di verificare i tipi di dati del dataset

In [ ]:
dataset.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,140.562500,55.683781,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242226,0
1,102.507812,58.882431,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393578,0
2,103.015625,39.341648,0.323328,1.051164,3.121238,21.744669,7.735822,63.171909,0
3,136.750000,57.178448,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593662,0
4,88.726562,40.672226,0.600866,1.123492,1.178930,11.468719,14.269573,252.567307,0


Possiamo notare che tutte le features tranne la classe sono reali, perchè sono il prodotto di calcoli statistici su dei fenomeni fisici e quindi intrinsecamente continui.
La classe di appartenenza invece è discreta, e visto che può assumere due valori possiamo dichiarare che il nostro problema consiste in una classificazione di tipo binario.

Un'altra analisi preliminare che possiamo eseguire consiste nell'analizzare dei dati statistici relativi ai nostri campi del dataset. Questa analisi estrapolerà questi valori:

- **count:** specifica il numero dei record presenti nel dataset
- **mean:** specifica la media dell'attributo calcolata per tutti i record
- **std:** specifica la deviazione standard dell'attributo
- **min:** specifica il valore minimo dell'attributo
- **25%:** il 25% dei record ha un valore minore di quello visualizzato (lower percentile)
- **50%:** il 50% dei record ha un valore minore di quello visualizzato (median percentile)
- **75%:** il 75% dei record ha un valore minore di quello visualizzato (upper percentile)
- **max:** specifica il valore massimo dell'attributo

In [ ]:
dataset.describe()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
count,17592.000000,17592.000000,17592.000000,17592.000000,17592.000000,17592.000000,17592.000000,17592.000000,17592.000000
mean,110.997955,46.545223,0.481985,1.793053,12.727508,26.407417,8.288137,104.641624,0.093167
std,25.751160,6.860444,1.071820,6.217242,29.629618,19.548374,4.513248,106.549126,0.290675
min,5.812500,24.772041,-1.876011,-1.791886,0.213211,7.370432,-3.139270,-1.976976,0.000000
25%,100.888672,42.377201,0.027163,-0.188526,1.927258,14.439169,5.745987,34.513126,0.000000
50%,115.050781,46.952198,0.223479,0.198906,2.810619,18.488600,8.421281,82.797821,0.000000
75%,127.023438,51.028583,0.474458,0.934944,5.501881,28.552639,10.694819,139.202213,0.000000
max,192.617188,98.778908,8.069522,68.101624,223.392136,110.642212,34.539845,1191.000854,1.000000


###Grafici
//TODO: Rivedere la lettura dei grafici!!

####Istogramma

In [ ]:
feature_histogram(dataset = dataset, title = "Distribuzione delle classi", feature = "target_class")

Da quel che possiamo vedere il dataset che abbiamo è molto sbilanciato. Per ogni record di classe "1" esistono 10 record di classe "0".

Questo sbilanciamento potrebbe portare il modello predittivo ad avere dei bias, portandolo a predirre più frequentemente la classe con più campioni solamente perchè è stata più presente in fase di train.

Gli Istogrammi sono molto utili per vedere la distribuzione delle classi per ogni feature in relazione al suo valore.
Tramite uno slider è possibile visualizzare delle features diverse.

In [82]:
import functools
import ipywidgets as widgets
pulsar = dataset[dataset['target_class'] == 1]
not_pulsar = dataset[dataset['target_class'] == 0]

#slider = widgets.SelectionSlider(
#    options=[col for col in dataset.columns if (col != 'target')],
#    value=dataset.columns[0],
#    disabled=False,
#    continuous_update=False,
#    orientation='horizontal',
#    readout=True
#)
#slider.observe(functools.partial(on_slider_change,dataset = dataset))
#display(slider)
import sys

create_slider_controlled_histogram(dataset.columns,pulsar,not_pulsar, "mean")

[' Mean of the integrated profile', ' Mean of the DM-SNR curve']


In [61]:
create_slider_controlled_histogram(dataset.columns,pulsar,not_pulsar,"deviation")

[' Standard deviation of the integrated profile', ' Standard deviation of the DM-SNR curve']
<class 'numpy.ndarray'>


In [72]:
create_slider_controlled_histogram(dataset.columns,pulsar,not_pulsar,"kurtosis")

[' Excess kurtosis of the integrated profile', ' Excess kurtosis of the DM-SNR curve']
<class 'numpy.ndarray'>


In [73]:
create_slider_controlled_histogram(dataset.columns,pulsar,not_pulsar,"skewness")

[' Skewness of the integrated profile', ' Skewness of the DM-SNR curve']
<class 'numpy.ndarray'>


A causa di un limite dello strumento, che consente un upload limitato di dati, ho dovuto dividere il grafico in 4. Ho optato per separare i grafici per metodo statistico applicato alle due misurazioni.

ho usato una normalizzazione percentuale che mi mostra, per ogni classe, la distribuzione percentuale dei valori. L'ho fatto a causa dell'elevata differenza numerica tra le due classi, che non mi permetteva di mostrare correttamente i valori assoluti.

Si può notare che la distribuzione dei valori "non pulsar" tende ad essere la maggior parte delle volte una distribuzione Gaussiana, e ogni tanto una distribuzione di Poisson.

La distribuzione dei valori "pulsar" tende ad essere più costante e disordinata, evitando una concentrazione di valori. Questo deriva dalle nostre supposizioni iniziali, che indicavano come univoci i pattern di ogni pulsar.

Questa differenza può essere significativa in fase di classificazione, perchè le varie features tendono a sovrapporsi poche volte, ciò implica che potenzialmente tutte le classi possono essere utili per determinare la classe.

####Boxplot

In [85]:
create_slider_controlled_Boxplot(dataset.columns,pulsar,not_pulsar, "mean")

[' Mean of the integrated profile', ' Mean of the DM-SNR curve']


In [86]:
create_slider_controlled_Boxplot(dataset.columns,pulsar,not_pulsar, "deviation")

[' Standard deviation of the integrated profile', ' Standard deviation of the DM-SNR curve']


In [87]:
create_slider_controlled_Boxplot(dataset.columns,pulsar,not_pulsar, "kurtosis")

[' Excess kurtosis of the integrated profile', ' Excess kurtosis of the DM-SNR curve']


In [88]:
create_slider_controlled_Boxplot(dataset.columns,pulsar,not_pulsar, "skewness")

[' Skewness of the integrated profile', ' Skewness of the DM-SNR curve']


Possiamo notare come tutti i parametri siano rilevanti nella classificazione. Nessuno di loro ha una distribuzione media troppo simile alla controparte della classe opposta. 

####Matrice di correlazione

Una matrice di correlazione è una tabella che mostra i coefficienti di correlazione tra insiemi di variabili. Ogni variabile casuale ( $X_i$ ) nella tabella è correlata con ciascuno degli altri valori nella tabella ( $X_j$ ); questo permette di vedere quali coppie hanno la più alta correlazione. La correlazione si riferisce a qualsiasi associazione statistica, ma nell'uso comune del termine si indica quanto due variabili siano vicine ad avere una relazione lineare l'una con l'altra.

Nel grafico sottostante abbiamo utilizzato la correlazione di Pearson, che misura la correlazione lineare tra due variabili statistiche X e Y, ritornando un valore compreso tra -1 e 1.
- Se il valore è vicino a -1 vuol dire che le due variabili hanno una forte correlazione negativa, cioè al crescere della prima decresce la seconda;
- Se il valore è vicino a 1 vuol dire che le due variabili hanno una forte correlazione, cioè al crescere della prima crescerà anche la seconda;
- Se il valore è vicino allo 0 le due variabili sono scorrelate

Questo valore viene calcolato così: $Corr_{i,j} = \frac{cov(X,Y)}{\sigma_x\sigma_y}$

dove cov è la covarianza tra i due valori e $\sigma$ è lo scarto quadratico medio.

In [90]:
import pandas as pd

corr = dataset.corr(method='pearson')
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
Mean of the integrated profile,1.00,0.55,-0.88,-0.74,-0.30,-0.31,0.24,0.14,-0.68
Standard deviation of the integrated profile,0.55,1.00,-0.52,-0.54,0.01,-0.05,0.03,0.03,-0.37
Excess kurtosis of the integrated profile,-0.88,-0.52,1.00,0.95,0.41,0.43,-0.34,-0.22,0.79
Skewness of the integrated profile,-0.74,-0.54,0.95,1.00,0.41,0.42,-0.33,-0.21,0.71
Mean of the DM-SNR curve,-0.30,0.01,0.41,0.41,1.00,0.80,-0.62,-0.36,0.40
Standard deviation of the DM-SNR curve,-0.31,-0.05,0.43,0.42,0.80,1.00,-0.81,-0.58,0.49
Excess kurtosis of the DM-SNR curve,0.24,0.03,-0.34,-0.33,-0.62,-0.81,1.00,0.92,-0.39
Skewness of the DM-SNR curve,0.14,0.03,-0.22,-0.21,-0.36,-0.58,0.92,1.00,-0.26
target_class,-0.68,-0.37,0.79,0.71,0.40,0.49,-0.39,-0.26,1.00


Possiamo notare che sia la skewness e la kurtosis (0,95 per l'integrated profile e 0,92 per la DM-SNR curve) che la mean e la standard deviation (0,55 per l'integrated profile e 0,80 per la DM-SNR curve) sono estremamente correlate tra loro.

Essendo che i nostri parametri sono ricavati dall'applicazione di funzioni statistiche su due grandezze fisiche, era aspettato trovare due blocchi di forte correlazione all'interno della matrice.

Possiamo notare come la kurtosis e la skewness of the integrated profile siano estremamente importanti per determinare la target_class, mentre la mean e la standard deviation della DM-SNR curve sono meno importanti, ma comunque molto rilevanti. Anche la mean of integrated profile ha una correlazione significativa (-0,68), cioè ha una forte correlazione inversa.

Notiamo che nessun attributo ha un indice di correlazione con la target_class vicino allo 0, ciò significa che sono tutti importanti nella determinazione della classe.

####Dendrogramma

###Suddivisione del dataset e normalizzazione dei dati

Per evitare "data leakage" è importante eseguire ogni trasformazione dopo la separazione dei dati. Il test set deve simulare dei dati reali acquisiti dopo l'addestramento del nostro modello, quindi non può essere utilizzato in nessun caso insieme al train set.

Molte volte i dataset contengono dei dati che variano molto rispetto alla media in termini di grandezza. Molti algoritmi di classificazione utilizzano la distanza euclidea per misurare la distanza tra due dati, quindi delle distanze troppo grandi possono causare dei problemi.

Per questo motivo, due features diverse possono influenzare in maniera diversa il classificatore in base all'unità di misura. Per esempio una features i cui valori sono dell'ordine delle migliaia (900, 5000, 2000) avrà un peso maggiore di una feature con valori nell'ordine delle unità (0,1, 10, 100) anche se in percentuale la variazione della prima risulterebbe essere minore rispetto alla seconda.

Per ovviare a questo problema utilizzeremo lo StandardScaler, che normalizzerà i nostri dati sottraendogli la media e dividendoli per lo scarto quadratico medio:
$x' = \frac{x - \mu_x}{\sigma_x}$

La normalizzazione verrà eseguita dopo la suddivisione dei dati, perchè il test set non deve essere influenzato in alcun modo dal train set e, eseguendo la normalizzazione sull'intero dataset, lo influenzerei tramite la media e la varianza del dataset intero.


In [ ]:
def normalize(X_set):
  scaler = StandardScaler(with_mean=True, with_std=True, copy=True)
  return scaler.fit_transform(X_set)

X_dataset = data.drop(['target_class'], axis=1)
y_data = data['target_class']
X_train, X_test, y_train, y_test = train_test_split(X_scaled_data,y_data, test_size=0.2, random_state=42, stratify=y_data)
X_train_norm = normalize(X_train)
X_test_norm = normalize(X_test)

<a name="PCA"></a>
##Principal Component Analysis

//TODO Vedere se è da fare prima PCA o prima il bilanciamento dei dataset 

https://stats.stackexchange.com/questions/7256/which-one-should-be-applied-first-data-sampling-or-dimensionality-reduction

https://stats.stackexchange.com/questions/31832/how-to-handle-data-imbalance-in-principal-component-analysis

##Bilanciamento Dataset 
//***TODO***: spiegare perchè ho scelto di usare pipeline e spiegare che si bilancia solo il train

Esistono diverse tecniche per bilanciare il dataset:
- **oversampling:** tecnica che consente di creare dei campioni appartenenti alla classe con meno record;
- **undersampling:** tecnica che consente di rimuovere dei campioni appartenenti alla classe con più record;
- **class weight:** tecnica che consente di bilanciare il dataset inserendo un parametro chiamato "weight" all'interno del dataset, che contiene i pesi delle varie classi del dataset. Il peso viene assegnato in base alla proporzione tra il numero di parametri della classe più grande e il numero di parametri della classe più piccola. (per esempio W = 1 per la classe "0" e W = 10 per la classe "1") 
Durante la fase di train i vari record influenzeranno il modello in proporzione al peso assegnatogli.
Questa è la soluzione meno invasiva, perché non modifica il dataset ma solamente il modo in cui i record agiscono sul modello.
-**pipeline**: questa tecnica consiste nell'applicare sia l'oversampling sul dataset più piccolo, che l'undersampling sul dataset più grande.

Per bilanciare questo dataset ho optato per la tecnica della pipeline, nella quale procederò a modificare i record di entrambe le classi.
//https://towardsdatascience.com/how-to-deal-with-imbalanced-data-in-python-f9b71aba53eb

Per capire quale sia il ratio migliore inserirò la procedura di train in un for, utilizzando per l'oversampling i valori [ 0.3 , 0.4 , 0.5 ] e per l'undersampling [ 0.7 , 0.6 , 0.5 ].
Sarà da selezionare la coppia con il valore ROC AUC maggiore.



In [ ]:
#dividere il dataset in train test e valid secondo un ratio da scegliere con la cross validation (?)
training_set = dataset

In [ ]:
#Bilanciamento dataset. da fare solo sul training set

x = training_set.iloc[:,:-1]
y = training_set.iloc[:,-1]

over_ratio = [0.3,0.4,0.5]
under_ratio = [0.7,0.6,0.5]
hi_score = 0
best_ratio = (0,0)
for o in over_ratio:
  for u in under_ratio:
    model = SVC()
    over = SMOTE(sampling_strategy=o)
    under = RandomUnderSampler(sampling_strategy=u)
    steps = [('over', over), ('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    scores = cross_val_score(pipeline, x, y, scoring='roc_auc', cv=5, n_jobs=-1)
    score = mean(scores)
    if score > hi_score:
      hi_score = score
      best_ratio = (o,u)
      print("cambio ratio {0} {1} ".format(o,u))
    print('SMOTE oversampling rate:%.1f, Random undersampling rate:%.1f , Mean ROC AUC: %.3f' % (o, u, score))
print("Il ratio che restituisce un punteggio maggiore è (oversampling: {0},undersampling: {1})".format(o,u))

//TODO: Spiegare cos'è SMOTE e Random Undersampling, come funzionma la pipeline, cos'è il ROC AUC score, cos'è e come mai ho scelto SVC come modello e quale ratio ho scelto. Magari fare un grafico con i vari score e provare con diversi modelli facendo una media degli score per ognuno di essi. 

//Sono da fare prima le trasformazioni orizzontali (bilanciamento dataset) o verticali (PCA)?

**SMOTE** (Synthetic Minority Oversampling Technique), è una tecnica

**Random Undersampling:**

**Pipeline:**

**ROC AUC Score:**

**SVC:**

##Classificazione